####
1. removing all zero scored records
2. combining the data
3. removing duplicates (keeping only test ones)

In [52]:
task_name='cosqa'
queries_corpus_dataset = load_dataset(f"CoIR-Retrieval/{task_name}-queries-corpus")
qrels_dataset = load_dataset(f"CoIR-Retrieval/{task_name}-qrels")

print('fetched data from hf')
corpus_data = queries_corpus_dataset['corpus']
query_data = queries_corpus_dataset['queries']
qrels_data_test = qrels_dataset['test']

fetched data from hf


In [53]:
corpus_data

Dataset({
    features: ['_id', 'partition', 'text', 'title', 'language', 'meta_information'],
    num_rows: 20604
})

In [59]:
corpus_data = load_dataset("vaishnavisha/cosqa_corpus_data")['train']
query_data = load_dataset("vaishnavisha/cosqa_query_data")['train']
qrels_data_test = load_dataset("vaishnavisha/cosqa_qrels_data_test")['train']

In [60]:
qrels_data_test

Dataset({
    features: ['query_id', 'corpus_id', 'score'],
    num_rows: 500
})

In [1]:
from coir.data_loader import get_tasks
from coir.evaluation import COIR
from coir.models import YourCustomDEModel
from coir.beir.retrieval import TrainRetriever
import pandas as pd
from datasets import load_dataset, Dataset, concatenate_datasets
from collections import defaultdict

In [2]:
queries_corpus_dataset = load_dataset(f"CoIR-Retrieval/cosqa-queries-corpus")
qrels_dataset = load_dataset(f"CoIR-Retrieval/cosqa-qrels")

In [3]:
corpus_data = queries_corpus_dataset['corpus']
query_data = queries_corpus_dataset['queries']
qrels_data_test = qrels_dataset['test']
qrels_df_test = qrels_data_test.to_pandas()
qrels_data_train = qrels_dataset['train']
qrels_data_valid = qrels_dataset['valid']
qrels_data = concatenate_datasets([qrels_data_train, qrels_data_valid, qrels_data_test])

In [4]:
test_query_ids = qrels_df_test['query_id'].to_list()

In [5]:
len(test_query_ids)

500

In [6]:
corpus_data

Dataset({
    features: ['_id', 'partition', 'text', 'title', 'language', 'meta_information'],
    num_rows: 20604
})

In [7]:
query_data

Dataset({
    features: ['_id', 'partition', 'text', 'title', 'language', 'meta_information'],
    num_rows: 20604
})

In [8]:
qrels_data

Dataset({
    features: ['query_id', 'corpus_id', 'score'],
    num_rows: 20604
})

In [9]:
qrels_df = qrels_data.to_pandas()
qrels_df = qrels_df[qrels_df["score"] == 1] # keeping only score 1
query_df = query_data.to_pandas()
corpus_df = corpus_data.to_pandas()

In [22]:
query_df = query_df.rename(columns={"_id": "query_id", "partition":"query_partition", "text": "query_text", "title": "query_title",  "language": "query_language", "meta_information": "query_meta_information"})
corpus_df = corpus_df.rename(columns={"_id": "corpus_id", "partition":"corpus_partition", "text": "corpus_text", "title": "corpus_title", "language": "corpus_language", "meta_information": "corpus_meta_information"})

merged = qrels_df.merge(query_df[["query_id", "query_partition", "query_text", "query_title", "query_language", "query_meta_information"]], on="query_id", how="left")

merged = merged.merge(corpus_df[["corpus_id", "corpus_partition","corpus_text", "corpus_title", "corpus_language", "corpus_meta_information"]], on="corpus_id", how="left")
merged = merged.sort_values(by="query_id", ascending=True)

In [26]:
merged[merged['corpus_text'] == 'def _convert_to_array(array_like, dtype):\n        """\n        Convert Matrix attributes which are array-like or buffer to array.\n        """\n        if isinstance(array_like, bytes):\n            return np.frombuffer(array_like, dtype=dtype)\n        return np.asarray(array_like, dtype=dtype)']

,query_id,corpus_id,score,query_partition,query_text,query_title,query_language,query_meta_information,corpus_partition,corpus_text,corpus_title,corpus_language,corpus_meta_information
0,q10,d10,1,train,1d array in char datatype in python,,,{'dummy_field': ''},train,"def _convert_to_array(array_like, dtype):\n ...",,PYTHON,{'dummy_field': ''}
2799,q9318,d9318,1,train,how to create a 2d array from 1d array using n...,,,{'dummy_field': ''},train,"def _convert_to_array(array_like, dtype):\n ...",,PYTHON,{'dummy_field': ''}


In [28]:
merged[merged['corpus_id'] == 'd10']

,query_id,corpus_id,score,query_partition,query_text,query_title,query_language,query_meta_information,corpus_partition,corpus_text,corpus_title,corpus_language,corpus_meta_information
0,q10,d10,1,train,1d array in char datatype in python,,,{'dummy_field': ''},train,"def _convert_to_array(array_like, dtype):\n ...",,PYTHON,{'dummy_field': ''}


In [31]:
merged[merged['corpus_id'] == 'd9318']

,query_id,corpus_id,score,query_partition,query_text,query_title,query_language,query_meta_information,corpus_partition,corpus_text,corpus_title,corpus_language,corpus_meta_information
0,q10,d9318,1,train,1d array in char datatype in python,,,{'dummy_field': ''},train,"def _convert_to_array(array_like, dtype):\n ...",,PYTHON,{'dummy_field': ''}
2799,q9318,d9318,1,train,how to create a 2d array from 1d array using n...,,,{'dummy_field': ''},train,"def _convert_to_array(array_like, dtype):\n ...",,PYTHON,{'dummy_field': ''}


In [30]:
last_corpus_ids = (
    merged
    .drop_duplicates(subset=["corpus_text"], keep="last")
    .set_index("corpus_text")["corpus_id"]
    .to_dict()
)

merged["corpus_id"] = merged["corpus_text"].map(last_corpus_ids)

merged = merged.reset_index(drop=True)

In [39]:
len(merged)

10020

In [43]:
query_df = merged[[
    "query_id", "query_partition", "query_text", "query_title", "query_language", "query_meta_information"
]].rename(columns={
    "query_id": "_id", "query_partition": "partition", "query_text": "text", 
    "query_title": "title", "query_language": "language", "query_meta_information": "meta_information"
})
query_data = Dataset.from_pandas(query_df.reset_index(drop=True))

# Corpus Data
corpus_df = merged[[
    "corpus_id", "corpus_partition", "corpus_text", "corpus_title", "corpus_language", "corpus_meta_information"
]].rename(columns={
    "corpus_id": "_id", "corpus_partition": "partition", "corpus_text": "text", 
    "corpus_title": "title", "corpus_language": "language", "corpus_meta_information": "meta_information"
})
corpus_data = Dataset.from_pandas(corpus_df.reset_index(drop=True))

# Qrels Data
qrels_df = merged[["query_id", "corpus_id", "score"]].drop_duplicates()
qrels_df_test = qrels_df[qrels_df["query_id"].isin(test_query_ids)]
qrels_data = Dataset.from_pandas(qrels_df.reset_index(drop=True))
qrels_data_test = Dataset.from_pandas(qrels_df_test.reset_index(drop=True))


In [61]:
query_data

Dataset({
    features: ['_id', 'partition', 'text', 'title', 'language', 'meta_information'],
    num_rows: 10020
})

In [48]:
corpus_data.push_to_hub("vaishnavisha/cosqa_corpus_data")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/11 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/vaishnavisha/cosqa_corpus_data/commit/dc56c5109ca530427fcfd21ad1b4d658c91d8cd5', commit_message='Upload dataset', commit_description='', oid='dc56c5109ca530427fcfd21ad1b4d658c91d8cd5', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/vaishnavisha/cosqa_corpus_data', endpoint='https://huggingface.co', repo_type='dataset', repo_id='vaishnavisha/cosqa_corpus_data'), pr_revision=None, pr_num=None)

In [49]:
qrels_data.push_to_hub("vaishnavisha/cosqa_qrels_data")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/11 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/vaishnavisha/cosqa_qrels_data/commit/9bb70630920d7bac4f811e2064f6f82801167b7c', commit_message='Upload dataset', commit_description='', oid='9bb70630920d7bac4f811e2064f6f82801167b7c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/vaishnavisha/cosqa_qrels_data', endpoint='https://huggingface.co', repo_type='dataset', repo_id='vaishnavisha/cosqa_qrels_data'), pr_revision=None, pr_num=None)

In [50]:
qrels_data_test.push_to_hub("vaishnavisha/cosqa_qrels_data_test")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/vaishnavisha/cosqa_qrels_data_test/commit/94d82a30ebfec85ba30fb0a680c2b5c255283c67', commit_message='Upload dataset', commit_description='', oid='94d82a30ebfec85ba30fb0a680c2b5c255283c67', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/vaishnavisha/cosqa_qrels_data_test', endpoint='https://huggingface.co', repo_type='dataset', repo_id='vaishnavisha/cosqa_qrels_data_test'), pr_revision=None, pr_num=None)

In [51]:
query_data.push_to_hub("vaishnavisha/cosqa_query_data")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/11 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/vaishnavisha/cosqa_query_data/commit/71a390af150552be131fa7607bad76135d079af1', commit_message='Upload dataset', commit_description='', oid='71a390af150552be131fa7607bad76135d079af1', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/vaishnavisha/cosqa_query_data', endpoint='https://huggingface.co', repo_type='dataset', repo_id='vaishnavisha/cosqa_query_data'), pr_revision=None, pr_num=None)